In [2]:
#IMPORTANDO OS PACOTE
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
#CARREGANDO AS BASES DE DADOS
os.chdir(r'data')
df_ipca = pd.read_excel('ipca_indicador.xlsx')
df_metas = pd.read_csv('Metas de inflação.csv', encoding='latin1', sep=';', nrows=25, decimal=',')
df_selic = pd.read_csv('selic.csv', encoding='latin1', sep=';', decimal=',', nrows=449, thousands='.')
df_expectativa_pib = pd.read_excel('expectativa_pib.xlsx')
df_expectativa_inflacao = pd.read_excel('expectativa_inflacao.xls', decimal=',', skiprows=1)
df_juros_natural = pd.read_excel('taxa_juros_natural.xlsx', decimal = ',')
df_ativ = pd.read_csv('IBC_BR.csv', decimal=',', encoding='latin1', sep=';')
df_prec = pd.read_csv('IC.csv', decimal=',', encoding='latin1', sep=';')
df_cambio = pd.read_csv('REER.csv')

In [ ]:
#CRIANDO A BASE DE DADOS COM O IBC-BR, o IC-Br e Taxa de câmbio ampla real (REER)
df_ativ.columns.values[0], df_ativ.columns.values[1] = 'date', 'IBC_BR'
df_ativ['date'] = pd.to_datetime(df_ativ['date'], format='%d/%m/%Y')
df_ativ['date'] = df_ativ['date'] + pd.offsets.MonthEnd(0)
df_prec.columns.values[0], df_prec.columns.values[1] = 'date', 'COMMODITIES'
df_prec['date'] = pd.to_datetime(df_prec['date'], format='%d/%m/%Y')
df_prec['date'] = df_prec['date'] + pd.offsets.MonthEnd(0)
df_cambio.columns.values[0], df_cambio.columns.values[1] = 'date', 'REER'
df_cambio['date'] = pd.to_datetime(df_cambio['date'], format='%Y-%m-%d')
df_cambio['date'] = df_cambio['date'] + pd.offsets.MonthEnd(0)
df_instrumentos = pd.merge(df_cambio, df_prec, on='date', how='left')
df_instrumentos = pd.merge(df_instrumentos, df_ativ, on='date', how='outer')
df_instrumentos.sort_values(by='date', inplace=True)
df_instrumentos[['REER', 'COMMODITIES', 'IBC_BR']] = df_instrumentos[['REER', 'COMMODITIES', 'IBC_BR']].pct_change().add_suffix('_pct_change')




#CRIANDO A BASE COM A TAXA DE JUROS NATURAL REAL
df_juros_natural.columns.values[0] = 'date'
df_juros_natural['date'] = pd.to_datetime(df_juros_natural['date'])
df_juros_natural = df_juros_natural.set_index('date')
df_juros_natural = df_juros_natural.resample('M').bfill()
df_juros_natural.reset_index(inplace=True)
df_juros_natural = pd.concat([pd.concat([df_juros_natural.iloc[[0]]] * 3), df_juros_natural.iloc[1:]])
df_juros_natural.reset_index(inplace=True, drop=True)
df_juros_natural.loc[0, 'date'] = df_juros_natural.loc[0, 'date'] + pd.DateOffset(months=-2)
df_juros_natural.loc[1, 'date'] = df_juros_natural.loc[1, 'date'] + pd.DateOffset(months=-1)
df_juros_natural['JUROS_NATURAL'] = df_juros_natural[['Exp Selic real 4a frente Focus', 'Hiato Band-pass', 
                                                      'Hiato Beveridge-Nelson', 'Hiato modelo semiestrutural']].mean(axis=1)
df_juros_natural['date'] = df_juros_natural['date'] + pd.offsets.MonthEnd(0)
df_juros_natural = df_juros_natural[['date','JUROS_NATURAL']]



#CRIANDO AS BASES COM AS EXPECTATIVAS
df_expectativa_inflacao.columns.values[0] = 'date'
df_expectativa_pib.columns.values[0] = 'date'
df_expectativa = df_expectativa_pib.merge(df_expectativa_inflacao, on='date', how='left')
df_expectativa['date'] = pd.to_datetime(df_expectativa['date'], format='%Y-%M-%d')
df_expectativa = df_expectativa[['date','EXP_PIB', 'EXP_INF']]
df_expectativa = df_expectativa.groupby(df_expectativa['date'].dt.to_period("M")).apply(lambda x: x.nlargest(1, 'date'))
df_expectativa.reset_index(drop=True, inplace=True)
df_expectativa['date'] = df_expectativa['date'].dt.to_period('M').dt.to_timestamp()
df_expectativa['date'] = df_expectativa['date'] + pd.offsets.MonthEnd(0)



#CRIANDO A BASE DE INFLAÇÃO
df_ipca[['mensal', 'anual']] = df_ipca[['mensal', 'anual']]
df_ipca['date'] = df_ipca['date'] + pd.offsets.MonthEnd(0)

#CRIANDO A META DE INFLAÇÃO MODIFICADA, CONSIDERANDO AS BANDAS
df_metas.columns.values[0], df_metas.columns.values[1] = 'date','meta_inflacao'
df_metas['meta_inflacao'] = df_metas['meta_inflacao'].astype(float)
df_metas['banda'] = df_metas['banda'].astype(float)
df_metas = df_metas.loc[df_metas.index.repeat(12)]
df_metas.reset_index(inplace=True, drop=True)
df_metas.date = pd.date_range(start='1999-01-01', end='2023-12-01', freq='MS')
df_metas['lim_inf'] = df_metas['meta_inflacao'] - df_metas['banda']
df_metas['lim_sup'] = df_metas['meta_inflacao'] + df_metas['banda']
df_metas['meta_modificada'] = ((12 - df_metas['date'].dt.month) / 12)*df_metas['meta_inflacao']  +  ((df_metas['date'].dt.month) / 12)*df_metas['meta_inflacao'].shift(-12)
df_metas['date'] = df_metas['date'] + pd.offsets.MonthEnd(0)


#CONSTRUINDO A BASE DA SELIC
df_selic.columns.values[0],  df_selic.columns.values[1] = 'date', 'selic'
df_selic['date'] = pd.to_datetime(df_selic['date'], format='%m/%Y')
df_selic['selic'] = df_selic['selic']
df_selic['date'] = df_selic['date'] + pd.offsets.MonthEnd(0)

#UNINDO AS BASES PARA CRIAR A BASE DE DADOS TOTAL
df_base_dados = df_ipca.merge(df_metas, on='date', how='left')
df_base_dados = df_base_dados.merge(df_selic, on='date', how='left')
df_base_dados = df_base_dados.merge(df_juros_natural, on='date', how='left')
df_base_dados = df_base_dados.merge(df_instrumentos, on='date', how='left')
df_base_dados = df_base_dados.set_index('date')



# Construindo os lags
num_lags = 4

for col in ['selic', 'IBC_BR', 'REER', 'COMMODITIES']:
    for i in range(1, num_lags + 1):
            df_base_dados[f'{col}_lag{i}'] = df_base_dados[col].shift(i)

# Filtrando para quando começou o regime de metas
df_base_dados = df_base_dados['1999-06-01':'2019-12-01']
df_base_dados.rename(columns={'anual':'acum_12_meses'}, inplace=True)
df_base_dados = df_base_dados.merge(df_expectativa, on='date', how='left')

#CONSTRUINDO O LAG ESPERADO DA INFLAÇÃO
df_base_dados['GAP_EXP_INF'] = df_base_dados['EXP_INF'] - df_base_dados['meta_modificada']


df_base_dados['gap_inflacao'] = df_base_dados['acum_12_meses'] - df_base_dados['meta_modificada']


for col in ['gap_inflacao']:
    for i in range(1, num_lags + 1):
            df_base_dados[f'{col}_lag{i}'] = df_base_dados[col].shift(i)




#CRIANDO O INDICADOR
num_columns = 60
for i in range(1, num_columns + 1):
    # Create new column names based on the iteration
    new_col_name = f'LOSS_{i}'
    
    # Perform the operation: A - B from the row above
    df_base_dados[new_col_name] = df_base_dados['gap_inflacao'].rolling(window=i).mean().shift(1)*abs(df_base_dados['gap_inflacao'].rolling(window=i).mean().shift(1))


df_base_dados.to_excel("base_dados_modelo (considerando somente a meta).xlsx", index=False)